# Assignment 6

### Part A

In [4]:
import numpy as np
import pandas as pd

Consider a DataFrame giving information about the price of a stock in 1 year, and the probability of that price occurring.

In [19]:
data = pd.DataFrame(data = {"Price in 1 Year" : [32, 43, 54],
                            "Probability" : [0.3, 0.4, 0.3]})

data2 = pd.DataFrame(data = {"Price in 1 Year" : [727, 41, 52, 65],
                            "Probability" : [0.4, 0.3, 0.2, 0.1]})
                    
data

Price in 1 Year  Probability
0               32          0.3
1               43          0.4
2               54          0.3

Write the function ```exp_return```, which calculates the expected return of the stock, given the current stock price ```P```.

In [21]:
def exp_return(data, P):
    # Write your code here
    expected_price = 0
    for row in range(len(data)):
        expected_price += data.iloc[row]['Price in 1 Year'] * data.iloc[row]['Probability']
    # exp_return(data, 40) should output 0.07500000000000002
    return expected_price/P - 1

In [4]:
exp_return(data, 40)

0.07499999999999996

Write the function ```volatility```, which calculates the expected return of the stock, given the current stock price ```P```.

In [23]:
def volatility(data, P):
    # Write your code here
    # Find the standard deviation based on the expected return:
    # SD(R) = sqrt{Var(R)} and Var(R) = \sum_R[P_R \times (R - E(R))^2]
    # where E(R) = \sum_R P_R \times R (from part A)
    ER = exp_return(data, P)
    var = 0
    
    for row in range(len(data)):
        var += data.iloc[row]['Probability'] * (pow((data.iloc[row]['Price in 1 Year']/P - 1) - ER, 2))
    vol = pow(var, 0.5)
    # volatility(data,40) should output 0.21301408404140795
    return vol

In [24]:
volatility(data2, 30)

11.079470504797

Write the function ```interval```, which calculates the 95% confidence interval for the expected return of the data. The corresponding Z score is 1.96. The function should return a list of size 2, with the first element representing the lower bound, and the second element representing the upper bound. 

In [7]:
def interval(data, P):
    # Write your code here
    # Use CI = R +/- 1.96 \times SE where SE = \frac{SD(R)}{\sqrt(N)} where N = 3 because we have 3 obvservations
    Z = 1.96
    N = len(data)
    sd = volatility(data, P)
    se = sd/pow(N, 0.5)
    er = exp_return(data, P)
    
    lower_bound = er - Z * se
    upper_bound = er + Z*se
    # interval(data, 40) should output [-0.16604812797447732, 0.3160481279744774]
    return [lower_bound,upper_bound]

In [8]:
interval(data, 40)

[-0.1660481279744774, 0.31604812797447734]

Write the function ```exp_return_capm```, which, given the beta of a security ```beta```, the return on the market ```market_return```,  and the risk free interest rate ```rf```, calculates the expected return of the security.

In [9]:
def exp_return_capm(beta, market_return, rf):
    # Write your code here
    # Use the formula: CAPM = r_f + \beta_i \times (E[R_{Mkt}] - r+f)
    
    capm = rf + beta * (market_return - rf)
    # exp_return_capm(-0.6,0.1, 0.02) should output -0.028
    return capm

In [10]:
exp_return_capm(-0.6,0.1, 0.02)

-0.028

### Part B

Write the function ```clean_data```, which will take a path to a csv file like the one provided. This function should remove all observations with a non number ```RET```, and remove all securities with (strictly) less than 500 observations. Your returned DataFrame should have 4 (non-index) columns, `PERMNO`, `date`, `CUSIP`, and `RET`.

In [5]:
def clean_data(path):
    # Write your code here
    df = pd.read_csv(path)

    # remove all observations with a non number RET
    df = df.dropna()
    df = df.loc[df['RET'] != 'B']
    df = df.loc[df['RET'] != 'C']

    groups = df.groupby(["CUSIP"], group_keys = False)
    filtered = groups.filter(lambda x: len(x) >= 500).reset_index().drop(['index'], axis = 1)
    
    # This should yield data with 403316 rows × 4 columns
    return filtered


In [7]:
path = '../../../Assignments/monthlycrsp_mod6.csv'
clean_data(path)

PERMNO        date     CUSIP        RET
0     10065  1970-01-30  00621210  -0.144375
1     10065  1970-02-27  00621210   0.105882
2     10065  1970-03-31  00621210   0.017857
3     10065  1970-04-30  00621210  -0.078947
4     10065  1970-05-29  00621210  -0.020952
..      ...         ...       ...        ...
631   10065  2022-08-31  00621210  -0.031250
632   10065  2022-09-30  00621210  -0.093342
633   10065  2022-10-31  00621210   0.074125
634   10065  2022-11-30  00621210   0.051757
635   10065  2022-12-30  00621210  -0.064350

[636 rows x 4 columns]

Write the function ```avg_return```, which will calculate the average return for each security. This function should take in a DataFrame ```data```, formatted like one outputted by the function ```clean_data```. The results should be returned in a DataFrame formatted like the one supplied.

Additionally, write the function ```security_volatilities```, which does the same thing as ```avg_return```, except it returns the volatilities of the returns of the securities.

In [11]:
partBdata1 = pd.DataFrame(data = {
    "PERMNO" : ["10000","10000","10000","20000","20000","20000","20000"],
    "date" : ["1970-01-30", "1970-02-30","1970-03-30","1970-01-30", "1970-02-30","1970-03-30","1970-04-30"],
    "CUSIP"  : [10000,10000,10000,20000,20000,20000,20000],
    "RET" : [0.2,0.4,-0.1,0.5,0.6,-0.2,-0.1]
})

In [15]:
def avg_return(data):
    # Write your code here
    # This should yield data with 679 rows × 2 columns
    data['RET'] = data['RET'].astype(float)

    groups = data.groupby(["CUSIP"], group_keys = False)
    cusip = []
    avg_ret = []

    for name, group in groups:
        cusip.append(name)
        avg_ret.append(group['RET'].mean())

    output = pd.DataFrame(data = {
        "CUSIP" : cusip,
        "Average Return" : avg_ret
    })
    
    return output

x = avg_return(partBdata1)
x.iloc[0]["Average Return"]

0.1666666666666667

In [14]:
#path = 'monthlycrsp_mod6.csv'
data = clean_data(path)

ret_data = avg_return(data)
print(len(ret_data))
ret_data.head()

679


CUSIP  Average Return
0  00036110        0.012326
1  00095710        0.011339
2  00105510        0.018398
3  00108M10        0.011101
4  00120410        0.012135

In [15]:
def security_volatilities(data):
    # Write your code here
    # This should yield data with 679 rows × 2 columns
    data['RET'] = data['RET'].astype(float)
    
    groups = data.groupby(["CUSIP"], group_keys = False)
    cusip = []
    vol = []
    
    # Use the Var(R) = \frac{1}{T - 1}sum_{t = 1}^T(R_t - \bar{R})^2 where we have already calculated \bar(R) is the question before
    
    returns = avg_return(data)
    data['RET'] = data['RET'].astype(float)

    for name, group in groups:
        cusip.append(name)
        var = 0
        count = 0
        for i in range(len(group)):
            var += pow(group.iloc[i]['RET'] - returns[returns['CUSIP'] == name]['Average Return'].values[0], 2)
            count += 1
        var = var/(count - 1)
        sd = pow(var, 0.5)
        vol.append(sd)

    output = pd.DataFrame(data = {
        "CUSIP" : cusip,
        "Volatility" : vol
    })
    
    return output

In [16]:
#path = 'monthlycrsp_mod6.csv'
data = clean_data(path)

vol_data = security_volatilities(data)
print(len(vol_data))
vol_data.head()

679


CUSIP  Volatility
0  00036110    0.122475
1  00095710    0.089881
2  00105510    0.099828
3  00108M10    0.171732
4  00120410    0.051050

Write the function ```correlation```, which will return the correlation between average return and volatility. This function takes two DataFrames, ```ret_data``` and ```vol_data```, which are the outputs of ```avg_return``` and ```security_volatilities``` respectively.

In [17]:
def correlation(ret_data, vol_data):
    # Write your code here
    # correlation(rets, vols) should yield 0.44837090727059614
    corr_df = {
        "Average Return": ret_data["Average Return"],
        "Volatility": vol_data["Volatility"]
    }
    
    corr = pd.DataFrame(corr_df)
    return corr.corr().iloc[0].iloc[1]

In [18]:
print(correlation(ret_data, vol_data))

0.44837090727059625
